In [ ]:
'''
talib
'''

In [1]:
import talib
import numpy as np

In [2]:
high = np.array([1.0, 2.0, 6.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0])
low = np.array([1.0, 2.0, 5.0, 6.0, 5.0, 4.0, 3.0, 2.0, 1.0])
close = np.array([1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
a = talib.ATR(high, low, close, 2)
a

array([       nan,        nan,  2.5      ,  3.25     ,  3.625    ,
        3.3125   ,  3.15625  ,  3.578125 ,  4.2890625])

In [ ]:
close = np.array([1000.0, 2.0, 1.0, 6.0, 6.0, 1.0, 6.0, 1.0, 1.0, 1.0, 1.0])
result = talib.RSI(close, 2)
result

In [ ]:
'''
tushare
'''

In [3]:
import tushare as ts
from datetime import datetime, timedelta

In [4]:
cons = ts.get_apis()

In [6]:
#df = ts.tick('rb1805', conn = cons, date = '2017-12-26', asset = 'X') 
#df = ts.tick('IF1801', conn = cons, date = '2017-06-22', asset = 'X') 
df = ts.tick('600380', conn = cons, date = '2018-01-02')

In [7]:
df

,datetime,price,vol,type
0,2018-01-02 09:30,11.19,753,2
1,2018-01-02 09:30,11.19,566,0
2,2018-01-02 09:30,11.20,623,0
3,2018-01-02 09:30,11.21,10,0
4,2018-01-02 09:30,11.23,3,0
5,2018-01-02 09:30,11.21,8,1
6,2018-01-02 09:30,11.21,37,1
7,2018-01-02 09:30,11.21,88,1
8,2018-01-02 09:30,11.23,19,0
9,2018-01-02 09:30,11.21,61,1


In [ ]:
df = ts.bar('rb1805', conn = cons, freq = 'D', start_date = '2017-01-01', end_date = '', asset = 'X')

In [ ]:
df.sort_index(inplace = True)
df

In [ ]:
a = df.ix[1]
the = a['date']
dt = the.to_datetime()
print dt
if dt.time() >= datetime.time(21, 0):
    dt = dt - timedelta(1)

print dt

In [ ]:
for index, row in df.iterrows():
    print row['date'], ' '*2, row['vol']

In [ ]:
from datetime import datetime

In [ ]:
from collections import namedtuple
abc = namedtuple('xyz', ['x', 'y'])
the = abc(6,2)
the.x

In [ ]:
import sys
#sys.getdefaultencoding()
#reload(sys)
#sys.setdefaultencoding('UTF-8')
a = sys.getdefaultencoding()
type(a)

In [ ]:
if isinstance('a', str):
    print 1
else:
    print 2

In [ ]:
import pymongo 

In [ ]:
c = pymongo.MongoClient('localhot', 270)
if c:
    print c
    
else:
    print 'no'

In [ ]:
from datetime import datetime
a = datetime(1900, 1, 1, 20).time()
print a